In [1]:
import numpy as np 

In [2]:
from sia import Pipeline
from sia.io import Metadata, read_edf, read_csv, write_csv
from sia.preprocessors import neurokit

In [3]:
target_labels = ['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat']
def encode_category(label):
    return {
        'label': np.isin(label, target_labels).astype(int)
    }

In [ ]:
Pipeline() \
    .data(
        read_edf(
            './data/ecg_raw/*.edf', 
            Metadata('./data/ecg_raw/TimeStamps_Merged.txt').on_regex(r'[0-9]{5}')
        )
    ) \
    .process(neurokit()) \
    .to(write_csv('./data/ecg_preprocessed/[0-9]{5}.csv'))

In [ ]:
Pipeline() \
    .data(read_csv('./data/ecg_preprocessed/*.csv', ['ECG_Clean', 'category'])) \
    .rename({'ECG_Clean': 'signal', 'category': 'label'}) \
    .process(encode_category) \
    .to(write_csv('./data/ecg_model/[0-9]{5}.csv'))